**Get place id**

In [4]:
from serpapi import GoogleSearch

SERPAPI_KEY = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"

def get_place_id(search_query):
    params = {
        "engine": "google_maps",
        "q": search_query,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        place = results["local_results"][0]
        name = place["title"]
        address = place["address"]
        place_id = place["place_id"]
        print(f"\n✅ Found place: {name} ({address})")
        return place_id
    except Exception as e:
        print("❌ Could not find place:", e)
        return None

def get_reviews_by_place_id(place_id):
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        reviews = results.get("reviews", [])
        print(f"\n📝 Found {len(reviews)} reviews:")
        for i, review in enumerate(reviews[:5]):
            print(f"\n🔸 Review {i+1}")
            print("⭐ Rating:", review["rating"])
            print("🕒 Time:", review["date"])
            print("🗣️ Review:", review["snippet"])
        return reviews
    except Exception as e:
        print("❌ Could not fetch reviews:", e)
        return []

if __name__ == "__main__":
    query = input("🔍 Enter location search (e.g., 'cafes in Bangalore'): ")
    place_id = get_place_id(query)
    if place_id:
        get_reviews_by_place_id(place_id)



✅ Found place: The Bangalore Cafe (4, Kengal Hanumanthaiah Rd, Bheemanna Garden, Shanti Nagar, Bengaluru, Karnataka 560027, India)

📝 Found 8 reviews:

🔸 Review 1
⭐ Rating: 4.0
🕒 Time: 2 months ago
🗣️ Review: Recently visited a cafe in Bangalore with my friend, and it turned out to be such a great experience! We ordered pizza, soup, and noodles — all purely vegetarian, and everything tasted really good. The place has a really cool vibe, with an ambience that's super cozy and perfect for chilling with friends or even family.They even have a big screen where they play live matches, which adds to the fun  especially if you’re visiting during game time! One thing to note is that the kitchen closes at 4 PM, but luckily we made it just in time and still got our food. The service was honestly amazing — the staff were really polite Totally recommend this spot if you’re looking for a nice veg cafe with chill vibes and good food in Bangalore!

🔸 Review 2
⭐ Rating: 4.0
🕒 Time: 2 months ago
🗣️ Re

**Review by places**

In [11]:
from serpapi import GoogleSearch
import requests

SERPAPI_KEY = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"
GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"


def get_place_id(search_query):
    params = {
        "engine": "google_maps",
        "q": search_query,
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        place = results["local_results"][0]
        name = place["title"]
        address = place["address"]
        place_id = place["place_id"]
        print(f"\n✅ Found place: {name} ({address})")
        return name, address, place_id
    except Exception as e:
        print("❌ Could not find place:", e)
        return None, None, None


def get_reviews_by_place_id(place_id):
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        reviews = results.get("reviews", [])
        print(f"\n📝 Found {len(reviews)} reviews:")
        for i, review in enumerate(reviews[:5]):
            print(f"\n🔸 Review {i+1}")
            print("⭐ Rating:", review["rating"])
            print("🕒 Time:", review["date"])
            print("🗣️ Review:", review["snippet"])
        return reviews
    except Exception as e:
        print("❌ Could not fetch reviews:", e)
        return []


def generate_vibe_summary(location, reviews):
    if not reviews:
        print("❌ No reviews provided.")
        return ""

    review_snippets = "\n".join(
        [f"{i+1}. \"{r['snippet']}\" (Rating: {r['rating']})" for i, r in enumerate(reviews[:10])]
    )

    prompt = f"""
You are VibeBot, a fun and friendly AI that summarizes the vibe of places based on real Google Maps reviews.

Place: {location}

Here are some user reviews:
{review_snippets}

Now write a short and engaging summary of the vibe of this place:
- Use emojis and a playful tone.
- Add vibe tags like #cozy, #lively, #chill, etc.
- Include 2–3 short citations from users like "One reviewer said..."
"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-8b-8192",  # ✅ Updated model here
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.8
    }

    try:
        response = requests.post(GROQ_URL, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        summary = result["choices"][0]["message"]["content"]
        print("\n🎯 Vibe Summary:\n")
        print(summary)
        return summary
    except requests.exceptions.HTTPError as http_err:
        print("❌ HTTP error occurred:", http_err)
        print("Response:", response.text)
        return ""
    except Exception as e:
        print("❌ Could not generate summary:", e)
        return ""


if __name__ == "__main__":
    query = input("🔍 Enter location search (e.g., 'cafes in Bangalore'): ")
    name, address, place_id = get_place_id(query)

    if place_id:
        reviews = get_reviews_by_place_id(place_id)
        full_location = f"{name}, {address}"
        if reviews:
            generate_vibe_summary(full_location, reviews)



✅ Found place: La Cafe (1st Sasha Building, 130, E Venkatasamy Rd, R.S. Puram, Coimbatore, Tamil Nadu 641002, India)

📝 Found 0 reviews:


In [9]:
from serpapi import GoogleSearch
import requests

SERPAPI_KEY = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"
GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

def get_places(query):
    params = {
        "engine": "google_maps",
        "q": query,
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    places = []
    try:
        for place in results.get("local_results", []):
            name = place.get("title", "Unknown")
            address = place.get("address", "Unknown")
            place_id = place.get("place_id", "")
            print(f"\n✅ Found place: {name} ({address})")
            if place_id:
                places.append((name, address, place_id))
    except Exception as e:
        print("❌ Could not get places:", e)
    return places

def get_reviews_by_place_id(place_id):
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        reviews = results.get("reviews", [])
        print(f"\n📝 Found {len(reviews)} reviews")
        return reviews
    except Exception as e:
        print("❌ Could not fetch reviews:", e)
        return []

def generate_vibe_summary(location, reviews):
    if not reviews:
        print("❌ No reviews provided.")
        return ""

    review_snippets = "\n".join(
        [f"{i+1}. \"{r['snippet']}\" (Rating: {r['rating']})" for i, r in enumerate(reviews[:10])]
    )

    prompt = f"""
You are VibeBot, a fun and friendly AI that summarizes the vibe of places based on real Google Maps reviews.

Place: {location}

Here are some user reviews:
{review_snippets}

Now write a short and engaging summary of the vibe of this place:
- Use emojis and a playful tone.
- Add vibe tags like #cozy, #lively, #chill, etc.
- Include 2–3 short citations from users like "One reviewer said..."
"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.8
    }

    try:
        response = requests.post(GROQ_URL, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        summary = result["choices"][0]["message"]["content"]
        print("\n🎯 Vibe Summary:\n")
        print(summary)
        return summary
    except requests.exceptions.HTTPError as http_err:
        print("❌ HTTP error occurred:", http_err)
        print("Response:", response.text)
        return ""
    except Exception as e:
        print("❌ Could not generate summary:", e)
        return ""

if __name__ == "__main__":
    query = input("🔍 Enter location search (e.g., 'cafes in Bangalore'): ")
    places = get_places(query)

    for name, address, place_id in places:
        reviews = get_reviews_by_place_id(place_id)
        if reviews:
            print(f"\n🎉 Summary for {name}, {address}")
            generate_vibe_summary(f"{name}, {address}", reviews)
        else:
            print(f"⚠️ No reviews found for {name}, skipping.")



✅ Found place: La Cafe (1st Sasha Building, 130, E Venkatasamy Rd, R.S. Puram, Coimbatore, Tamil Nadu 641002, India)

✅ Found place: Cardiff Cafe Cheranmanagar (Shop Site, No – 8021/19 at, near Tidal Park, Cheran ma Nagar, Coimbatore, Tamil Nadu 641004, India)

✅ Found place: Cafe retroo (2nd floor, 191/3-4, Raja Annamalai Rd, Jawahar Nagar, Saibaba Colony, Coimbatore, Tamil Nadu 641011, India)

✅ Found place: Café Totaram (Beejay center, 245/1, Race Course Rd, Race Course, Coimbatore, Tamil Nadu 641018, India)

✅ Found place: Sai masala cafe (51, Dr Radhakrishna St, opposite Icici bank, Kec Colony, Tatabad, Coimbatore, Tamil Nadu 641012, India)

✅ Found place: High For This (Floor 2, Grand The Hub, 1515/3A, Avinashi Rd, BR Puram, PKD Nagar, Coimbatore, Tamil Nadu 641004, India)

✅ Found place: Hifive Cafe (3, Sathy Rd, behind City Pazamudir, Ramanandha Nagar, Saravanampatti, Coimbatore, Tamil Nadu 641035, India)

✅ Found place: Café Tantos (30, Subramaniam Rd, R.S. Puram, Coimbatore,

In [13]:
from serpapi import GoogleSearch
import requests
import time
import re

SERPAPI_KEY = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"
GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

def retry_with_backoff(func, *args, max_retries=5, base_delay=1, **kwargs):
    """
    Retry a function with exponential backoff on HTTP 429 errors.
    """
    retries = 0
    while retries < max_retries:
        try:
            return func(*args, **kwargs)
        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                # Extract wait time from error message, e.g., "Please try again in 6.363s"
                error_message = http_err.response.json().get("error", {}).get("message", "")
                wait_time_match = re.search(r"Please try again in ([\d.]+)s", error_message)
                wait_time = float(wait_time_match.group(1)) if wait_time_match else base_delay * (2 ** retries)
                time.sleep(wait_time)
                retries += 1
            else:
                raise http_err
        except Exception as e:
            raise e
    print(f"⚠️ Failed to process request for {args[0]} after {max_retries} retries due to rate limits.")
    return ""

def get_places(query):
    params = {
        "engine": "google_maps",
        "q": query,
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    places = []
    try:
        for place in results.get("local_results", []):
            name = place.get("title", "Unknown")
            address = place.get("address", "Unknown")
            place_id = place.get("place_id", "")
            print(f"\n✅ Found place: {name} ({address})")
            if place_id:
                places.append((name, address, place_id))
    except Exception as e:
        print("❌ Could not get places:", e)
    return places

def get_reviews_by_place_id(place_id):
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        reviews = results.get("reviews", [])
        print(f"\n📝 Found {len(reviews)} reviews")
        return reviews
    except Exception as e:
        print("❌ Could not fetch reviews:", e)
        return []

def generate_vibe_summary(location, reviews):
    if not reviews:
        print("❌ No reviews provided.")
        return ""

    review_snippets = "\n".join(
        [f"{i+1}. \"{r['snippet']}\" (Rating: {r['rating']})" for i, r in enumerate(reviews[:10])]
    )

    prompt = f"""
You are VibeBot, a fun and friendly AI that summarizes the vibe of places based on real Google Maps reviews.

Place: {location}

Here are some user reviews:
{review_snippets}

Now write a short and engaging summary of the vibe of this place:
- Use emojis and a playful tone.
- Add vibe tags like #cozy, #lively, #chill, etc.
- Include 2–3 short citations from users like "One reviewer said..."
"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.8
    }

    def make_request():
        response = requests.post(GROQ_URL, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"]

    try:
        summary = retry_with_backoff(make_request)
        if summary:
            print("\n🎯 Vibe Summary:\n")
            print(summary)
        return summary
    except Exception as e:
        print(f"⚠️ Could not generate summary for {location}.")
        return ""

if __name__ == "__main__":
    query = input("🔍 Enter location search (e.g., 'cafes in Bangalore'): ")
    places = get_places(query)

    for name, address, place_id in places:
        reviews = get_reviews_by_place_id(place_id)
        if reviews:
            print(f"\n🎉 Summary for {name}, {address}")
            generate_vibe_summary(f"{name}, {address}", reviews)
        else:
            print(f"⚠️ No reviews found for {name}, skipping.")


✅ Found place: La Cafe (1st Sasha Building, 130, E Venkatasamy Rd, R.S. Puram, Coimbatore, Tamil Nadu 641002, India)

✅ Found place: Cardiff Cafe Cheranmanagar (Shop Site, No – 8021/19 at, near Tidal Park, Cheran ma Nagar, Coimbatore, Tamil Nadu 641004, India)

✅ Found place: Cafe retroo (2nd floor, 191/3-4, Raja Annamalai Rd, Jawahar Nagar, Saibaba Colony, Coimbatore, Tamil Nadu 641011, India)

✅ Found place: Café Totaram (Beejay center, 245/1, Race Course Rd, Race Course, Coimbatore, Tamil Nadu 641018, India)

✅ Found place: Sai masala cafe (51, Dr Radhakrishna St, opposite Icici bank, Kec Colony, Tatabad, Coimbatore, Tamil Nadu 641012, India)

✅ Found place: High For This (Floor 2, Grand The Hub, 1515/3A, Avinashi Rd, BR Puram, PKD Nagar, Coimbatore, Tamil Nadu 641004, India)

✅ Found place: Hifive Cafe (3, Sathy Rd, behind City Pazamudir, Ramanandha Nagar, Saravanampatti, Coimbatore, Tamil Nadu 641035, India)

✅ Found place: Café Tantos (30, Subramaniam Rd, R.S. Puram, Coimbatore,

In [22]:
from serpapi import GoogleSearch
import requests
import time
import re

SERPAPI_KEY = "bd0d819e093e6c8acf32d6aaab551208bb2f15b199966fa215daa790ba73c33d"
GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

def retry_with_backoff(func, *args, max_retries=5, base_delay=1, **kwargs):
    retries = 0
    while retries < max_retries:
        try:
            return func(*args, **kwargs)
        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                error_message = http_err.response.json().get("error", {}).get("message", "")
                wait_time_match = re.search(r"Please try again in ([\d.]+)s", error_message)
                wait_time = float(wait_time_match.group(1)) if wait_time_match else base_delay * (2 ** retries)
                time.sleep(wait_time)
                retries += 1
            else:
                raise http_err
        except Exception as e:
            raise e
    print(f"⚠️ Failed to process request after {max_retries} retries.")
    return ""

def get_places(query):
    params = {
        "engine": "google_maps",
        "q": query,
        "api_key": SERPAPI_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    
    places = []
    try:
        for place in results.get("local_results", []):
            name = place.get("title", "Unknown")
            address = place.get("address", "Unknown")
            place_id = place.get("place_id", "")
            if place_id:
                places.append((name, address, place_id))
    except Exception as e:
        print("❌ Could not get places:", e)
    return places

def get_reviews_by_place_id(place_id):
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "api_key": SERPAPI_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    try:
        return results.get("reviews", [])
    except Exception as e:
        return []

def generate_vibe_summary(location, reviews):
    if not reviews:
        return ""

    review_snippets = "\n".join(
        [f"{i+1}. \"{r['snippet']}\" (Rating: {r['rating']})" for i, r in enumerate(reviews[:10])]
    )

    prompt = f"""
You are VibeBot, a fun and friendly AI that summarizes the vibe of places based on real Google Maps reviews.

Place: {location}

Here are some user reviews:
{review_snippets}

Now write a short and engaging summary of the vibe of this place:
- Use emojis and a playful tone.
- Add vibe tags like #cozy, #lively, #chill, etc.
- Include 2–3 short citations from users like "One reviewer said..."
"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-8b-8192",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.8
    }

    def make_request():
        response = requests.post(GROQ_URL, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"]

    try:
        summary = retry_with_backoff(make_request)
        return summary
    except Exception:
        return ""

if __name__ == "__main__":
    query = input("🔍 Enter location search (e.g., 'cafes in Bangalore'): ")
    places = get_places(query)

    for name, address, place_id in places:
        reviews = get_reviews_by_place_id(place_id)
        if reviews:
            summary = generate_vibe_summary(f"{name}, {address}", reviews)
            if summary:
                print(f"\n✅ {name}, {address}")
                print(summary)
